In [1]:
spark

In [2]:
machineEventsDF = sqlContext.read.format("csv").load("/FileStore/tables/1mr287ha1493790179044/part_00000_of_00001-79fc9.csv")

In [3]:
display(machineEventsDF.limit(5))

In [4]:
machineEventsDF = machineEventsDF.drop('_c3')

In [5]:
display(machineEventsDF.limit(5))

In [6]:
machineCapacityDF = machineEventsDF.drop('_c0', '_c2').dropDuplicates()

In [7]:
machineCapacityDF.count()

In [8]:
machineCapacityDF = machineCapacityDF.groupBy(['_c4', '_c5']).count()

In [9]:
display(machineCapacityDF)

In [10]:
from pyspark.sql.functions import ceil
dailyEventsDF = machineEventsDF.drop('_c1', '_c4', '_c5').filter("_c0 > 600 * 1000000").filter("_c0 < 9223372036854775807").withColumn('temp', (machineEventsDF._c0 / 1000000 - 600) / 86400)
dailyEventsDF = dailyEventsDF.withColumn('day', ceil(dailyEventsDF.temp))

In [11]:
dailyEventsDF.count()

In [12]:
dailyEventsDF = dailyEventsDF.groupBy(['_c2', 'day']).count()

In [13]:
sorted(dailyEventsDF.collect())

In [14]:
dailyEventsPDDF = dailyEventsDF.toPandas()

In [15]:
dailyEventsPDDF = dailyEventsPDDF.pivot(index='day', columns='_c2', values='count')

In [16]:
dailyEventsPDDF

In [17]:
import matplotlib.pyplot as plt
dailyEventsFig = dailyEventsPDDF.plot.area(stacked=False)
display(dailyEventsFig.figure)

In [18]:
dailyEventsPDDF.sum()

In [19]:
dailyEventsPDDF.mean()

In [20]:
dailyEventsPDDF.mean() / 12622

In [21]:
eventsCountDF = machineEventsDF.drop('_c0', '_c4', '_c5').groupBy('_c1').pivot('_c2', [0, 1, 2]).count().na.fill(0)

In [22]:
display(eventsCountDF)

In [23]:
eventsCountDF.describe().show()

In [24]:
event0CountDF = eventsCountDF.groupBy('0').count().sort('0')

In [25]:
display(event0CountDF)

In [26]:
event1CountDF = eventsCountDF.groupBy('1').count().sort('1')

In [27]:
event2CountDF = eventsCountDF.groupBy('2').count().sort('2')

In [28]:
event0CountPDDF = event0CountDF.toPandas()
event1CountPDDF = event1CountDF.toPandas()
event2CountPDDF = event2CountDF.toPandas()

In [29]:
eventCountFig = event0CountPDDF.plot(x='0', y='count', color='b', label='0', logx=True, logy=True)
event1CountPDDF.plot(x='1', y='count', color='g', label='1', logx=True, logy=True, ax=eventCountFig)
event2CountPDDF.plot(x='2', y='count', color='r', label='2', logx=True, logy=True, ax=eventCountFig)
display(eventCountFig.figure)